In [ ]:
%matplotlib inline
import sys
sys.path.append("..")

# Analyzing videos of a particle

In this tutorial, we will train a network to extract the intensity of a particle regardless of focus. The network will be trained to predict on videos of centered particles moving in and out of focus randomly.

## 1 Setup

Imports needed for this example.

In [ ]:
from deeptrack.scatterers import Sphere
from deeptrack.optics import Fluorescence
from deeptrack.noises import Offset, Poisson
from deeptrack.aberrations import SphericalAberration, HorizontalComa, VerticalComa
from deeptrack.augmentations import Augmentation, FlipLR, FlipUD, FlipDiagonal
from deeptrack.sequences import Sequence, Sequential
from deeptrack.features import Feature
from deeptrack.generators import Generator
from deeptrack.models import RNN


import numpy as np
import matplotlib.pyplot as plt

## 2.1 Define the particle

We want to generate a video where the position of the particle changes over time. The x-y position of the particle will be roughly centered in each frame. This is to simulate a tracked particle centered by cropping a region of interest around it for each frame. The z position, however, will follow a Brownian motion with a random diffusion constant. 

To allow these properties to change in the video, we define two functions that return the value of these properties for each frame.

In [ ]:
def get_position():
    return 32 + np.random.randn(2) * 0.5

def get_z(previous_value, diffusion_constant, dt=1/30):
    return np.clip(previous_value + np.random.randn() * np.sqrt(diffusion_constant * dt) * 1e7, -60, 60)

## 2.2 Set initial values of the properties

The properties passed to create the instance of `Sphere` are used for the first frame of the video.

* `position` is initialized with the same function as that used for each subsequent frame.
* `z` is initialized as a normally distributed number centered around zero.
* `intensity` is constant 1. The intensity will be varied in a later step.
* `radius` is a random number between 200nm and 1000nm
* `position_unit` is set to pixels
* `upsample` is 4, increasing the resolution of the sphere.


In [ ]:
particle_feature = Sphere(
    position=get_position,
    z=lambda: np.random.randn() * 20,
    intensity=1,
    radius=lambda: 200e-9  + np.random.rand()*800e-9,
    diffusion_constant=lambda: (1 + np.random.rand() * 9) * 1e-12,
    position_unit="pixel",
    upsample=4
)

## 2.3 Make it sequential

To tell `particle_feature` how to change for each frame in a video, we call `Sequential` with the two functions defined in 2.1.

In [ ]:
sequential_particle = Sequential(particle_feature, z=get_z, position=get_position)

## 3.1 Define the full feature series

We define an optical device with some aberration. It is hard for the network to learn with heavy aberration. To mitigate this, the network will start training with very little aberration, and slowly increase the aberration as the network trains. 

To achieve this, we define a dummy property, `epoch`, which will return a number which increases by one each time the property is updated, up to `200`. The property `coefficient`, in turn, returns a normally distributed random number, with a standard deviation that scales with `epoch`. This way, the more times the property is updated, the more the optical device is aberrated.

In [ ]:
epoch_cap = 200
get_coefficient = lambda epoch: np.random.randn() * 0.1 * epoch / epoch_cap

spherical_aberration = SphericalAberration(epoch=iter(range(1, epoch_cap + 1)), 
                                           coefficient=get_coefficient)

horizontal_coma = HorizontalComa(epoch=iter(range(1, epoch_cap + 1)), 
                                 coefficient=get_coefficient)

vertical_coma = VerticalComa(epoch=iter(range(1, epoch_cap + 1)), 
                             coefficient=get_coefficient)

aberrations = spherical_aberration + horizontal_coma + vertical_coma

optics = Fluorescence(
    NA=0.7,
    magnification=10,
    resolution=(1e-6, 1e-6, 1e-6),
    wavelength=633e-9,
    output_region=(0, 0, 64, 64),
    pupil=aberrations
)

The on-camera captured intensity scales with the radius cubed times the property `intensity`. This means that the apparent intensity is much more dependent on the radius of the object than the intensity of each voxel. To remedy this, we define and apply a normalization feature to the `Sphere`. `NormalizeSum` ensure that the sum of all pixels in the input is equal to `scale`. Thus, the apparent intensity will only depend on the property `intensity` of the sphere.

In [ ]:
class NormalizeSum(Feature):
    def get(self, image, scale=1, **kwargs):
        return image / np.sum(image) * scale

## 3.2 Bring it together

We define the full feature series by imaging the sequential particle and the normalization feature with `optics`. `Sequence(imaged_article, sequence_length=10)`, in turn, creates a feature that resolves videos of length 10.  

In [ ]:
imaged_particle = optics(sequential_particle + NormalizeSum(scale=100)) 

imaged_particle_sequence = Sequence(imaged_particle, sequence_length=10)  

imaged_particle_sequence.update()
imaged_particle_sequence.plot()

## 3.3 Vary the intensity

Varying the intensity of the particle is equivalent to scaling the output image. This allows us to define an augmentation that converts a video of a particle of any intensity to any other intensity.

We additionally define the method `update_properties`. This method is unique to the class `Augmentation`, and it lets us update the properties of the video in accordance with the augmentation used. Here, it updates all instances of the `intensity` property in the video by multiplying it by the `multiplier` property of the feature.

In [ ]:
class AugmentIntensity(Augmentation):
    def get(self, image, multiplier, number_of_updates, **kwargs):
        return image * multiplier
    
    def update_properties(self, image, multiplier, **kwargs):
        for prop in image.properties:
            if "intensity" in prop:
                prop["intensity"] *= multiplier

## 3.4 Augment the videos

For augmentation we will use a 8-way mirroring augmentation, followed by the aforementioned intensity augmentation. Noise is added after the augmentation to further diversify the generated dataset.

In [ ]:
augmented_particle_sequence = FlipUD(FlipLR(FlipDiagonal(imaged_particle_sequence)))
augmented_particle_sequence = AugmentIntensity(augmented_particle_sequence, 
                                               multiplier=lambda: 1 + np.random.rand() * 9,
                                               updates_per_reload=2)

augmented_particle_sequence += Offset(offset=0.1) + Poisson(snr=200)

In [ ]:
augmented_particle_sequence.update()
augmented_particle_sequence.plot()

## 4.1 Define the model

The model used will be `RNN`, which is a Recurrent Neural Network. The model acts like a convolutional neural network applied to each step of the video, followed by a LSTM layer, which merges all the information into a single output. RNNs can model complicated temporal behavours of videos of arbitrary length.

In [ ]:
model = RNN(input_shape=(None, 64, 64, 1), 
            rnn_layers_dimensions=(32, 32),
            dense_layers_dimensions=(32, 32),
            number_of_outputs=1, 
            loss="mse")


## 4.2 Define the label and the generator

The label function extracts the property `intensity` from the first frame of the video. The generator is a standard generator with `ndim` set to 5 to account for it resolving videos instead of images.

In [ ]:
def get_intensity_of_particle(video):
    return video[0].get_property("intensity")

generator = Generator().generate(
    augmented_particle_sequence, 
    get_intensity_of_particle, 
    batch_size=4,
    ndim=5
)

## 5. Train the model

We train the model for 30 epochs, consisting of 100 batches eachs. This may take some time (upwards of half an hour).

Note that some versions of Tensorflow do not correctly handle videos like these correctly. If you see an error such as `GPU sync failed` or any error pointing to there being too little memory, consider trying Tensorflow version 1.14, or tf-nightly.

In [ ]:
model.fit(generator, epochs=200, steps_per_epoch=100)

## 6. Validate the model

We validate the correctness of the model by generating 200 videos and creating a scatter plot of predicted intensity vs. actual intensity.

In [ ]:
generator = Generator().generate(
    augmented_particle_sequence, 
    get_intensity_of_particle, 
    batch_size=1,
    ndim=5
)

predicted_intensity = []
true_intensity = []
for _ in range(200):
    batch, labels = next(generator)
    batch = np.asarray(batch)

    prediction = model.predict(batch)
    predicted_intensity.append(prediction[0])
    true_intensity.append(labels[0])
        
plt.scatter(true_intensity, predicted_intensity)
plt.plot([1, 10], [1, 10])
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Predicted intensity vs true intensity")
plt.xlabel("True intensity")
plt.ylabel("Predicted intensity")
plt.show()